<a href="https://colab.research.google.com/github/i40-Tools/I40KG-Embeddings/blob/master/Similarity_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# I40 standards landscape similarity analysis using embeddings

## Overview

In this notebook, we show the similarity analysis between Industry 4.0 Standards. 
To do so, we create embeddings about the Industry 4.0 Standards Knowledge Graph (I40KG) developed by [Grangel-Gonzales et. al.](https://www.researchgate.net/publication/318208930_The_Industry_40_Standards_Landscape_from_a_Semantic_Integration_Perspective)

The embeddings are located here: [I40 Embeddings](https://github.com/i40-Tools/I40KG-Embeddings/tree/master/logs_sto)

## Initial Configurations
First, let's import the required libraries to perform the similarity analysis.

In [5]:
%ls

I40KG-Embeddings/  sample_data/


In [2]:
!git clone https://github.com/i40-Tools/I40KG-Embeddings.git

Cloning into 'I40KG-Embeddings'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 82 (delta 34), reused 37 (delta 9), pack-reused 0
Unpacking objects: 100% (82/82), done.


In [11]:
input_corpus='/content/I40KG-Embeddings/sto/sto.nt'
g = Graph()
g.parse(input_corpus, format="nt")


<Graph identifier=N3a1734ef38d7449492b41311461988dc (<class 'rdflib.graph.Graph'>)>

In [0]:
import scipy
from scipy import spatial
import numpy as np
import math
import json
from scipy.spatial.distance import cdist

### Define similarity function
We are going to us cosine distance to measure the similarity between two embeddings

In [0]:
#function to calculate cosine distance 
def cosine_similarity(vec1,vec2):
    sum11, sum12, sum22 = 0, 0, 0
    for i in range(len(vec1)):
        x = vec1[i]; y = vec2[i]
        sum11 += x*x
        sum22 += y*y
        sum12 += x*y
    return sum12/math.sqrt(sum11*sum22)

## Similarity among I40 Standarization Frameworks
In this section we show the analysis of similarity among standarization frameworks

In [8]:
!pip3 install rdflib

     |████████████████████████████████| 348kB 4.8MB/s 
     |████████████████████████████████| 51kB 18.7MB/s 


In [0]:
import json
from rdflib import Graph
import pprint

In [0]:
g = Graph()
g.parse(".../I40KG-Embeddings-master/sto/sto.nt", format="nt")
    
len(g) # prints 2
 #check printing of the graph    
'''for stmt in g:
    pprint.pprint(stmt)'''

#query to get the framework/standard from the sto.nt file
#to get standards we have to replace sto:StandardizationFramework by sto:Standard in the query
#we can get standards of the frameworks as well by just changing the query

qres = g.query(
    """PREFIX owl: <http://www.w3.org/2002/07/owl#>
       PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
       PREFIX sto: <https://w3id.org/i40/sto#>
    
    select ?s where {
            ?s rdf:type sto:StandardizationFramework .
            } limit 1000""")
    
       
#to get the corresponding embeddings of the frameworks/standards from the json file 
with open(".../I40KG-Embeddings-master/logs_sto/entities_to_embeddings.json",'rb') as f:
    array = json.load(f)
new_dict = {}
for row in qres:
    for key,value in array.items():
        if key == "%s" % row:
            tem = key
            new_dict[tem] = array[key] 
            print(new_dict)

#to put the frameworks/standards with their corresponding embeddings in a file            
with open('output.json','w') as f:
    # this would place the entire output on one line
    # use json.dump(new_dict, f, indent=4) to "pretty-print" with four spaces per indent
    json.dump(new_dict, f)
    
    
#to read the file containing standards/frameworks along with their embeddings   
with open('output.json', 'r') as f:
    array = json.load(f)
    
#compare each standard/framework with all the other standards/frameworks to find cosine similarity
result = {}
for key,value in array.items():
    temp,tempDict= 0,{}
    for keyC,valueC in array.items():
        if keyC == key:
            continue
        temp = scipy.spatial.distance.cosine(value,valueC) #send the values of the standards/frameworks to cosine similarity function
        tempDict[keyC] = temp
        val1 = min(tempDict, key=tempDict.get)
    res = {}
    res[val1] = tempDict[val1]
    #print (res)
    result[key]= res
    #result[key]= tempDict
        
#print the standards/frameworks along with their similar standards/frameworks and their similarity distance
print(result)